<a href="https://colab.research.google.com/github/khush2520/Word-Sense-Disambiguation/blob/main/finallll_WSD_sem_cor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [155]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from google.colab import drive
drive.mount('/content/drive')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [156]:
import xml.etree.ElementTree as ET

In [157]:

xml_file = 'drive/MyDrive/semcor.data.xml'
gold_key = 'drive/MyDrive/semcor.gold.key.txt'
test_xml_file = 'drive/MyDrive/ALL.data.xml'
test_gold_key = 'drive/MyDrive/ALL.gold.key.txt'

In [158]:
import os

file_path = 'drive/MyDrive/dataset.json'  # Replace 'your_file_path_here' with the path to your file

if os.path.exists(file_path):
    # Get the size of the file in bytes
    file_size_bytes = os.path.getsize(file_path)

    # Convert bytes to megabytes
    file_size_mb = file_size_bytes / (1024 * 1024)

    print(f"The size of the file is approximately {file_size_mb:.2f} MB")
else:
    print("File not found.")


The size of the file is approximately 340.03 MB


In [159]:
# def find_corpus(file_name):
#     with open(file_name, 'r') as file:
#         lines = file.read()
#         a = (len(lines)//1)
#         print(a)
#         print(lines.count("/corpus"))
#         i = 0
#         while "<corpus" in lines:
#           i +=1
#           index = lines.index("<corpus")
#           # print(lines[index:index+a//100])
#           index2 = lines.index("</corpus")



#           # print(index,index2)
#           corpus = lines[index:index2]
#           print(index,index2,lines[index2-10:index2+10])
#           lines = lines[0:index] + lines[index2+10:]
#           with open(f'drive/MyDrive/temp{i}.xml', 'w') as file:

#               file.write('<?xml version="1.0" encoding="UTF-8" ?>\n')
#               file.write('<root>\n')
#               file.write(corpus)
#               file.write('</corpus>\n')
#               file.write('</root>\n')


# find_corpus(xml_file)


In [160]:
# print(lines)

In [161]:
# def divide_corpus(file_name):
#     with open(file_name, 'r') as file:
#         lines = file.read()
#         a = (len(lines)//10)
#         print(lines.count("/sentence"))
#         i = 0
#         while "<sentence" in lines:
#           i +=1
#           index = lines.index("<sentence")
#           # print(lines[index:index+a//100])
#           index2 = a + lines[a:].index("</sentence")
#           if len(lines) < 2* a:
#               index2 = lines.rindex("</sentence")



#           # print(index,index2)
#           corpus = lines[index:index2]
#           print(index,index2,lines[index2-10:index2+12])
#           lines = lines[0:index] + lines[index2+12:]
#           with open(f'drive/MyDrive/tempdd{i}.xml', 'w') as file:

#               file.write('<?xml version="1.0" encoding="UTF-8" ?>\n')
#               file.write('<root>\n')
#               file.write(corpus)
#               file.write('</sentence>\n')
#               file.write('</root>\n')


# divide_corpus("drive/MyDrive/temp2.xml")


In [162]:
dataset= {}
ins = 0

In [163]:

def process(file):
    # root = ET.fromstring(corpus)
    n = 0
    global ins
    tree = ET.parse(file)
    root = tree.getroot()
    # print(type(root))
    for sentence in root.findall('.//sentence'):
        # print(f"Sentence ID: {sentence.attrib['id']}")
        sent = ""
        n+=1
        for element in sentence:
                sent = sent + " " + element.text
        for element in sentence:
            if element.tag in ['instance']:
                  ins += 1
                  lemma = element.attrib.get('lemma')
                  id = element.attrib.get('id')
                  pos = element.attrib.get('pos')
                  if lemma not in dataset:
                    dataset[lemma] = []
                  dataset[lemma].append([id,lemma, sent,pos,element.text])

        # print(sent)

    print(len(dataset))
    return n


In [164]:
process(xml_file)
# for i in range(1,10):
#     print(process(f'drive/MyDrive/tempdd{i}.xml'))
print(ins)

20399
226036


In [165]:
import json
with open('drive/MyDrive/semcordataset.json', 'w') as file:
    json.dump(dataset, file, indent=2)

In [169]:
with open('drive/MyDrive/semcordataset.json', 'r') as file:
    dataset = json.load(file)
i = 0
for d in dataset:
  i += 1
  if i > 12:
    break
# print(d, dataset[d])

In [170]:
lines = []
with open(gold_key, 'r') as file:
    lines = file.readlines()

Y_values = {}
processed_data = []
for line in lines:
    words = line.strip().split()
    processed_data.append(words[1])
    Y_values[words[0]] = words[1]
#     print(item)

print(len(processed_data))
print(len(set(processed_data)))
# print(Y_values)

226036
33316


In [171]:

for lemma in dataset:
  for j in dataset[lemma]:
    if (j[0] == None):
          print(lemma,j)
    y = Y_values[j[0]]
    pos = y.index('%')
    j.append(y[pos:pos+8])


In [172]:
file_path = test_gold_key
# lines = []
# with open(test_gold_key, 'r') as file:
#     lines = file.readlines()
# # print(lines[0:9])

# test_Y_values = {}
# test_processed_data = []
# for line in lines:
#     words = line.strip().split()
#     test_processed_data.append(words[1])
#     test_Y_values[words[0]] = words[1]
#     # pos = words[0].find('%')

#     # processed_data.append(words[0][:pos])

# test_Y = test_processed_data
# # print(test_Y_values)
# print(len(test_processed_data))
# print(len(set(test_processed_data)))
# print(Y_values)lines = []
with open(test_gold_key, 'r') as file:
    lines = file.readlines()

test_Y_values = {}
test_processed_data = []
for line in lines:
    words = line.strip().split()
    pos = line.index(" ")
    line = line[pos:]
    test_processed_data.append(line)
    test_Y_values[words[0]] = line
#     print(item)
test_Y = test_processed_data

print(len(test_processed_data))
print(len(set(test_processed_data)))
# print(Y_values)

7253
3612


In [173]:

test_tree = ET.parse(test_xml_file)
test_root = test_tree.getroot()
# print(type(root))
test_dataset = {}
for sentence in test_root.findall('.//sentence'):
    # print(f"Sentence ID: {sentence.attrib['id']}")
    sent = ""
    for element in sentence:
            sent = sent + " " + element.text
    for element in sentence:
            if element.tag in ['instance']:
              lemma = element.attrib.get('lemma')
              id = element.attrib.get('id')
              pos = element.attrib.get('pos')
              if lemma not in test_dataset:
                test_dataset[lemma] = []
              test_dataset[lemma].append([id,lemma, sent,pos,element.text])
            # print(f"{element.text} Lemma: {lemma}, POS: {pos}")
    # print(sent)

print(len(test_dataset))
# print(features)


2659


In [174]:
for lemma in test_dataset:
  for j in test_dataset[lemma]:
    y = test_Y_values[j[0]]
    pos = y.index('%')
    j.append(y[pos:])


In [175]:
from nltk.corpus import wordnet

def print_all_senses(word):
    synsets = wordnet.synsets(word)
    if synsets:
        print(f"All senses of '{word}':")
        for synset in synsets:
            print(f"{synset.name()}:  Sense Key: {synset.lemmas()[-1].key()} {synset.definition()}" )
    else:
        print(f"No senses found for '{word}'.")


In [176]:
from nltk.corpus import wordnet as wn
POS  = {'n':'NOUN', 'a': 'ADJ','s': 'ADJ', 'v': 'VERB', 'r':'ADV'}
def add_dataset_from_wordnet(word):
    word2 = word.replace('_',' ')
    word2 = word2.replace('-',' ')
    # print("word", word)
    synsets = wn.synsets(word)
    if synsets:
      for word_synset in synsets:
        lemma = word_synset.lemmas()[0].name()
        example_sentence = word_synset.examples()[0] if word_synset.examples() else word_synset.definition()

        p = word_synset.pos()
        pos = POS[p]

        # Find the word form in the example sentence
        word_form = None
        if example_sentence is not None:
          word_forms = example_sentence.split()
          for wf in word_forms:
            wf_synsets = wn.synsets(wf)
            for wf_synset in wf_synsets:
                wf_lemma = wf_synset.lemmas()[0].name()
                # print(wf_l)
                if lemma == wf_lemma:
                  word_form = wf
                  break;
            if word_form:
              break
        for sense in word_synset.lemmas():
          sense_key = sense.key()
          posi = sense_key.index('%')
          if sense_key.startswith(word+'%'):
            posi = sense_key.index('%')
            break

        if word_form is None:
          word_form = word
        if word not in dataset:
          dataset[word] = []
        dataset[word].append([None, lemma, example_sentence, pos, word_form, sense_key[posi:]])
        # print('hi', [lemma, example_sentence, pos, word_form, sense_key])
    else:
        print('hi',word, [None])



In [177]:
words = list(test_dataset.keys())

for lemma in words:
    add_dataset_from_wordnet(lemma)

In [178]:
import json
with open('drive/MyDrive/final_dataset.json', 'w') as file:
    json.dump(dataset, file, indent=2)


In [179]:
with open('drive/MyDrive/final_dataset.json', 'r') as file:
    dataset = json.load(file)

In [180]:
#Navie Bayes
words = list(test_dataset.keys())
total_accuracy = 0
lc = 0
for lemma in words:
    if lemma not in dataset:
      add_dataset_from_wordnet(lemma)
      # print(lemma)
      continue

    X_train = [row[1:-1] for row in dataset[lemma]]
    y_train = [row[-1] for row in dataset[lemma]]


    X_test = [row[1:-1] for row in test_dataset[lemma]]
    y_test = [row[-1] for row in test_dataset[lemma]]
    vectorizer = DictVectorizer(sparse=False)
    X_train_vectorized = vectorizer.fit_transform([dict(zip(['lemma', 'sent', 'pos', 'element.text'], x)) for x in X_train])

    # X_train_vectorized = vectorizer.fit_transform([dict(zip(['pos', 'element.text'], x)) for x in X_train])
    naive_bayes_classifier = MultinomialNB()

    naive_bayes_classifier.fit(X_train_vectorized, y_train)

    # X_test_vectorized = vectorizer.transform([dict(zip(['pos', 'element.text'], x)) for x in X_test])
    X_test_vectorized = vectorizer.transform([dict(zip(['lemma', 'sent', 'pos', 'element.text'], x)) for x in X_test])
    predictions = naive_bayes_classifier.predict(X_test_vectorized)
    # accuracy = naive_bayes_classifier.score(X_test_vectorized, y_test)

    if len(set(y_train)) < 2:
        # if list(set(test_y))[0] != list(set(train_y))[0]:
        #     print(lemma, test_y, list(set(train_y))[0])

        lc += len(y_test)
        total_accuracy += 1 * len(y_test)
        continue
    # print("Accuracy:", accuracy)
    correct = 0
    total = 0
    for p, g in zip(predictions, y_test):
      total += 1
      if p in g:
        correct += 1
    accuracy = correct/total
    lc+= len(X_test)
    total_accuracy += accuracy * len(X_test)
print(total_accuracy/lc)


0.6554529160347442


In [186]:
#KNN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction import DictVectorizer

words = list(test_dataset.keys())
total_accuracy = 0
lc = 0

for lemma in words:
    if lemma not in dataset:
        print(lemma)
        continue

    X_train = [row[1:-1] for row in dataset[lemma]]
    y_train = [row[-1] for row in dataset[lemma]]

    X_test = [row[1:-1] for row in test_dataset[lemma]]
    y_test = [row[-1] for row in test_dataset[lemma]]

    if len(set(y_train)) < 2:
        lc += len(y_test)
        # total_accuracy += 1 * len(X_test)
        continue

    vectorizer = DictVectorizer(sparse=False)
    X_train_vectorized = vectorizer.fit_transform([dict(zip(['lemma', 'sent', 'pos', 'element.text'], x)) for x in X_train])

    knn_classifier = KNeighborsClassifier(n_neighbors=(max(2,len(y_train)//3)))  # You can adjust n_neighbors as needed

    knn_classifier.fit(X_train_vectorized, y_train)

    X_test_vectorized = vectorizer.transform([dict(zip(['lemma', 'sent', 'pos', 'element.text'], x)) for x in X_test])

    predictions = knn_classifier.predict(X_test_vectorized)
    correct = 0
    total = 0
    for p, g in zip(predictions, y_test):
      total += 1
      if p in g:
        correct += 1
    accuracy = correct/total



    lc += len(X_test)
    total_accuracy += accuracy * len(X_test)

print(total_accuracy / lc)


0.5891355301254654


In [182]:
#AdaBoost
from sklearn.ensemble import AdaBoostClassifier
from sklearn.feature_extraction import DictVectorizer

words = list(test_dataset.keys())
total_accuracy = 0
lc = 0
i = 0
for lemma in words:

    if lemma not in dataset:
        print(lemma)
        continue

    X_train = [row[1:-1] for row in dataset[lemma]]
    y_train = [row[-1] for row in dataset[lemma]]

    X_test = [row[1:-1] for row in test_dataset[lemma]]
    y_test = [row[-1] for row in test_dataset[lemma]]

    if len(set(y_train)) < 2:
        lc += len(y_test)
        total_accuracy += 1 * len(X_test)
        continue

    vectorizer = DictVectorizer(sparse=False)
    X_train_vectorized = vectorizer.fit_transform([dict(zip(['lemma', 'sent', 'pos', 'element.text'], x)) for x in X_train])

    adaboost_classifier = AdaBoostClassifier(n_estimators=50, random_state=42)  # You can adjust n_estimators as needed

    adaboost_classifier.fit(X_train_vectorized, y_train)

    X_test_vectorized = vectorizer.transform([dict(zip(['lemma', 'sent', 'pos', 'element.text'], x)) for x in X_test])

    predictions = adaboost_classifier.predict(X_test_vectorized)
    # accuracy = adaboost_classifier.score(X_test_vectorized, y_test)
    correct = 0
    total = 0
    for p, g in zip(predictions, y_test):
      total += 1
      if p in g:
        correct += 1
    accuracy = correct/total
    # if 1: #list (predictions) != y_test:
    #   print(lemma,list(predictions),y_test,accuracy)
    #   if list(predictions) != y_test:
    #     print (X_train)
    #     print (y_train)
    #     print (X_test)
    #   i += 1
    #   if i > 10:
    #     break
    # print accura
    lc += len(X_test)
    total_accuracy += accuracy * len(X_test)
    print(lemma, accuracy,"\n")

print(total_accuracy / lc)


art 0.0 

peculiar 1.0 

english 0.5 

most 1.0 

peculiarity 1.0 

unintelligible 0.0 

rest 1.0 

world 0.0 

tailor 1.0 

england 1.0 

scene 0.0 

evoke 0.0 

rural 1.0 

lovely 1.0 

ancient 1.0 

stone 1.0 

church 0.6666666666666666 

stand 0.7142857142857143 

field 0.375 

sound 0.5 

bell 1.0 

cascade 0.0 

tower 1.0 

call 0.5555555555555556 

faithful 0.0 

evensong 0.0 

parishioner 1.0 

stop 0.2857142857142857 

chat 1.0 

door 0.4 

member 0.7777777777777778 

here 0.5714285714285714 

always 1.0 

man 0.9523809523809523 

woman 1.0 

pull 0.3333333333333333 

rope 1.0 

attach 0.0 

same 0.5 

first 0.7 

also 1.0 

discordant 0.0 

modern 1.0 

note 0.5 

churchgoer 1.0 

enjoy 1.0 

peal 1.0 

cool 1.0 

autumn 1.0 

evening 1.0 

other 0.9696969696969697 

britain 1.0 

set 0.6 

once 0.5 

own 1.0 

band 0.0 

ringer 1.0 

herald 1.0 

sunday 1.0 

morning 1.0 

service 0.6 

now 0.0 

only 0.4117647058823529 

local 0.875 

remain 0.5 

today 0.5 

live 0.5714285

In [184]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

# Assuming your test_dataset and dataset are defined elsewhere

words = list(test_dataset.keys())
total_accuracy = 0
lc = 0

for lemma in words:
    if lemma not in dataset:
        add_dataset_from_wordnet(lemma)
        # print(lemma)
        continue

    X_train = [row[1:-1] for row in dataset[lemma]]
    y_train = [row[-1] for row in dataset[lemma]]

    X_test = [row[1:-1] for row in test_dataset[lemma]]
    y_test = [row[-1] for row in test_dataset[lemma]]
    if len(set(y_train)) < 2:
        lc += len(y_test)
        total_accuracy += 1 * len(y_test)
        continue

    vectorizer = DictVectorizer(sparse=False)
    X_train_vectorized = vectorizer.fit_transform(
        [dict(zip(['lemma', 'sent', 'pos', 'element.text'], x)) for x in X_train]
    )

    # Initialize Logistic Regression (MaxEnt) classifier
    maxent_classifier = LogisticRegression(max_iter=1000)

    maxent_classifier.fit(X_train_vectorized, y_train)

    X_test_vectorized = vectorizer.transform(
        [dict(zip(['lemma', 'sent', 'pos', 'element.text'], x)) for x in X_test]
    )

    predictions = maxent_classifier.predict(X_test_vectorized)


    correct = 0
    total = 0

    for p, g in zip(predictions, y_test):
        total += 1
        if p in g:
            correct += 1

    accuracy = correct / total
    lc += len(X_test)
    total_accuracy += accuracy * len(X_test)

print(total_accuracy / lc)


0.660140631462843
